In [ ]:
#pip install -r requirements.txt

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from datetime import datetime, timedelta
import re

In [3]:
st.set_page_config(
    page_title="FED Sentiment Analysis Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded")

OUTPUT_DIR = "dashboard_data"
PROCESSED_DIR = "processed_data"
ANALYSIS_DIR = "analysis_data"

for directory in [OUTPUT_DIR, PROCESSED_DIR, ANALYSIS_DIR]:
    os.makedirs(directory, exist_ok=True)

In [5]:
# custom CSS
st.markdown("""
<style>
.main-header {
    font-size: 2.5rem;
    color: #1E3A8A;
    text-align: center;
    margin-bottom: 1rem;
}
.sub-header {
    font-size: 1.5rem;
    color: #1E3A8A;
    margin-top: 2rem;
    margin-bottom: 1rem;
}
.chart-container {
    background-color: #f8f9fa;
    padding: 1rem;
    border-radius: 5px;
    margin-bottom: 2rem;
}
.insight-box {
    background-color: #e9f7ef;
    padding: 1rem;
    border-left: 5px solid #27ae60;
    margin-bottom: 1rem;
}
</style>
""", unsafe_allow_html=True)

2025-04-28 20:21:55.959 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()